# Train Code for Convolutional Neural Networks  
CNNs are trained using this notebook

## The Data

In [ ]:
import utils
reload(utils)

In [ ]:
(X_train, Y_train), (X_test, Y_test) = utils.load_data()

In [ ]:
print X_train.shape
print X_test.shape
print Y_train.shape
print Y_test.shape

## The Model  
We use a 'typical' CNN with Max-Pooling layers to predict classes in CIFAR-10 dataset.  

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

Optionally, install [Hualos](https://github.com/fchollet/hualos) and use it to visualize the loss function live.  

In [ ]:
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')

### Define the model

In [ ]:
pool_size = (2, 2)
strides = (2, 2)

model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(3,32,32)))  # Layer 1
model.add(Convolution2D(32, 3, 3, activation='relu'))  # Layer 2
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))  # Layer 3
model.add(Dropout(0.25))  # Layer 4

model.add(Convolution2D(64, 3, 3, activation='relu'))  # Layer 5
model.add(Convolution2D(64, 3, 3, activation='relu'))  # Layer 6
model.add(MaxPooling2D(pool_size=pool_size, strides=strides))  # Layer 7
model.add(Dropout(0.25))  # Layer 8

model.add(Flatten())  # Layer 9

model.add(Dense(512, activation='relu'))  # Layer 10
model.add(Dropout(0.5))  # Layer 11
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

Or load saved model

In [ ]:
model = utils.load_model('model-big-50')

### Train  the model

In [ ]:
data_augmentation = True
batch_size = 32
nb_epoch = 50
rounds = 2

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1, 
              show_accuracy=True)
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(X_train, rounds=rounds)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch, show_accuracy=True,
                        nb_worker=1)

In [ ]:
model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1, 
          show_accuracy=True)

### Test the model

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test, show_accuracy=True)
print "Loss:", loss
print "Accuracy:", accuracy

### Save the model

In [ ]:
utils.save_model(model, 'model-5x5-3x3-3-50')